In [3]:
from __future__ import print_function
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import ELU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


# 5 классов(5 людей), размер фотографии 48х48
 
num_classes = 5
img_rows, img_cols = 48, 48
batch_size = 16



train_data_dir = './faces/train/'
validation_data_dir = './faces/validation/'



# Data Augmentation 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      shear_range=0.3,
      zoom_range=0.3,
      width_shift_range=0.4,
      height_shift_range=0.4,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 



train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

Found 2841 images belonging to 5 classes.
Found 1078 images belonging to 5 classes.


In [48]:
# Архитектура модели 
# 7 блоков


model = Sequential()


# 1-й блок
model.add(Conv2D(32, (3, 3), padding = 'same', kernel_initializer="he_normal",
                 input_shape = (img_rows, img_cols, 3)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding = "same", kernel_initializer="he_normal", 
                 input_shape = (img_rows, img_cols, 3)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


# 2-й блок
model.add(Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


# 3-й блок
model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


# 4-й блок
model.add(Conv2D(256, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


# 5-й блок
model.add(Flatten())
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


# 6-й блок
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


# 7-й блок
model.add(Dense(num_classes, kernel_initializer="he_normal"))
model.add(Activation("softmax"))

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 48, 48, 32)        896       
_________________________________________________________________
activation_44 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_40 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_45 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_41 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 24, 24, 32)       

In [49]:
# TRAINING PART


from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
                     

# Checkpoint
checkpoint = ModelCheckpoint("test2_face_recognition_friends_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

# Learning Rate 
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)


callbacks = [checkpoint, reduce_lr]


# Компилируюм модель, Adam optimizer
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.01),
              metrics = ['accuracy'])


nb_train_samples = 2841
nb_validation_samples = 1078
epochs = 10

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 177 steps, validate for 67 steps
Epoch 1/10
176/177 [============================>.] - ETA: 0s - loss: 1.0547 - accuracy: 0.6155
Epoch 00001: val_loss improved from inf to 1.77262, saving model to test2_face_recognition_friends_vgg.h5
177/177 [==============================] - 28s 156ms/step - loss: 1.0516 - accuracy: 0.6159 - val_loss: 1.7726 - val_accuracy: 0.4552
Epoch 2/10
176/177 [============================>.] - ETA: 0s - loss: 0.7041 - accuracy: 0.7544
Epoch 00002: val_loss did not improve from 1.77262
177/177 [==============================] - 27s 152ms/step - loss: 0.7033 - accuracy: 0.7547 - val_loss: 2.2645 - val_accuracy: 0.6856
Epoch 3/10
176/177 [============================>.] - ETA: 0s - loss: 0.6061 - accuracy: 0.7949
Epoch 00003: val_loss improved from 1.77262 to 1.72874, saving model to test2_face_recognition_friends_vgg.h5
177/177 [==============================] - 26s 146ms/step - loss: 0.6062 - accuracy:

In [4]:
# Присвоение лэйблов

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
class_labels

{0: 'Aidyn', 1: 'Chandler', 2: 'Joey', 3: 'Pheobe', 4: 'Rachel'}

In [1]:
# Загрузка модели

from tensorflow.keras.models import load_model

classifier = load_model('test2_face_recognition_friends_vgg.h5')

In [2]:
from os import listdir
from os.path import isfile, join
import os
import cv2
import numpy as np
import dlib
from tensorflow.keras.preprocessing.image import img_to_array


#5 классов 
face_classes = {0: 'Aidyn', 1: 'Chandler', 2: 'Joey', 3: 'Pheobe', 4: 'Rachel'}


# Запись предикшена на видео
def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)
    
    
margin = 0.2
img_size = 64



#детектор лица
detector = dlib.get_frontal_face_detector()



# Тестовое видео
cap = cv2.VideoCapture('aaa.MOV')


# Детекция лица на видео 
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    preprocessed_faces = []           
 

    input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = np.shape(input_img)
    detected = detector(frame, 1)
    faces = np.empty((len(detected), img_size, img_size, 3))
    
    
    if len(detected) > 0:
        
        for i, d in enumerate(detected):
            
            # Позиции лица и навешиваем прямоугольник 
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            xw1 = max(int(x1 - margin * w), 0)
            yw1 = max(int(y1 - margin * h), 0)
            xw2 = min(int(x2 + margin * w), img_w - 1)
            yw2 = min(int(y2 + margin * h), img_h - 1)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        
        
            #Encoding лица
            face =  frame[yw1:yw2 + 1, xw1:xw2 + 1, :]
            face = cv2.resize(face, (48, 48), interpolation = cv2.INTER_AREA)
            face = face.astype("float") / 255.0
            face = img_to_array(face)
            face = np.expand_dims(face, axis=0)
            preprocessed_faces.append(face)

        # prediction лица полученные выше
        face_labels = []
        for i, d in enumerate(detected):
            preds = classifier.predict(preprocessed_faces[i])[0]
            face_labels.append(face_classes[preds.argmax()])
        
        # Вставляем результаты в видео
        for i, d in enumerate(detected):
            label = "{}".format(face_labels[i])
            draw_label(frame, (d.left(), d.top()), label)

    cv2.imshow("Face Identifier", frame)
    if cv2.waitKey(1) == 13: 
        break

cap.release()
cv2.destroyAllWindows()      